In [1]:
#| default_exp workflow

In [2]:
import msprime
import sys

print(f"msprime version being used: {msprime.__version__}")
print(f"Python executable: {sys.executable}")

msprime version being used: 1.3.4
Python executable: /bin/python


In [3]:
#|export

import jax
from chewc.config import SimConfig
from chewc.structs import SimParam
from chewc.popgen import quick_haplo
from chewc.state import init_state_from_founders
import jax.numpy as jnp


# 1. Define static simulation configuration
config = SimConfig(
    n_chr=10,
    ploidy=2,
    max_pop_size=1000,
    n_loci_per_chr=100,
    n_generations=50,
    n_select=50,
    population_size=101  # invalid value, not multiple
)

# 2. Create a master random key
key = jax.random.PRNGKey(42)
key, founder_key = jax.random.split(key)

# 3. Generate the founder population (runs on host)
founder_pop, genetic_map = quick_haplo(
    key=founder_key,
    n_ind=100,
    n_chr=config.n_chr,
    n_loci_per_chr=config.n_loci_per_chr,
    max_pop_size=config.max_pop_size,
    ploidy=config.ploidy
)

# 4. Define simulation parameters (e.g., genetic maps, trait info)
sp = SimParam(
    gen_map=genetic_map,
    ploidy=config.ploidy,
    # traits, var_e, etc. would be defined here
)

# 5. Initialize the dynamic simulation state (the carry for lax.scan)
# This uses your existing `init_state_from_founders` function.
key, state_key = jax.random.split(key)
initial_state = init_state_from_founders(
    key=state_key,
    founder_pop=founder_pop,
    sp=sp,
    config=config
)

print(f"Initial write position: {initial_state.write_pos}")
print(f"Next available ID: {initial_state.next_id}")
print(f"Founder population active: {jnp.sum(initial_state.is_active)}")

# `initial_state` is now ready to be passed to your JIT'd `generation_step`
# within a `lax.scan`.

Initial write position: 101
Next available ID: 100
Founder population active: 100


/mnt/c/Users/cltng/gdrive/chewc/chewc/state.py:72: UserWarning: max_pop_size (1000) is not a multiple of population_size (101). Automatically increasing max_pop_size to 1010 for a clean ring buffer.
  warnings.warn(


In [1]:
import jax
import jax.numpy as jnp

# Assume the new meiosis module is available
from chewc.meiosis import produce_offspring 
from chewc.config import SimConfig
from chewc.structs import SimParam
from chewc.popgen import quick_haplo
from chewc.state import init_state_from_founders
from chewc.trait import add_trait_a, set_pheno_h2

# 1. Define static simulation configuration
config = SimConfig(
    n_chr=10,
    ploidy=2,
    max_pop_size=1000,
    n_loci_per_chr=100,
    n_generations=50,
    n_select=50,
    population_size=100,
)

# 2. Create and split master random key
key = jax.random.PRNGKey(42)
key, founder_key, trait_key, pheno_key, state_key, selection_key, meiosis_key = jax.random.split(key, 7)

# 3. Generate the founder population
founder_pop, genetic_map = quick_haplo(
    key=founder_key,
    n_ind=100,
    n_chr=config.n_chr,
    n_loci_per_chr=config.n_loci_per_chr,
    max_pop_size=config.max_pop_size,
    ploidy=config.ploidy,
)

# 4. Define initial simulation parameters
sp = SimParam(
    gen_map=genetic_map,
    ploidy=config.ploidy,
    # Set a default recombination parameter `v` for the Gamma process
    recomb_params=(2.6, 0.0, 0.0)
)

# 5. Add an additive trait to the simulation parameters
print("--- Defining Trait ---")
sp = add_trait_a(
    key=trait_key,
    founder_pop=founder_pop,
    sim_param=sp,
    n_qtl_per_chr=10,
    mean=jnp.array([0.0]),
    var=jnp.array([1.0]),  # Target *genetic* variance
)
print(f"Trait added with {sp.traits.n_loci} QTLs.")

# 6. Set phenotypes using a target narrow-sense heritability (h2)
print("\n--- Setting Initial Phenotypes with h2 ---")
h2 = 0.4
founder_pop_with_pheno = set_pheno_h2(
    key=pheno_key, pop=founder_pop, sp=sp, h2=h2
)

# 7. Initialize the dynamic simulation state
print("\n--- Initializing Simulation State ---")
initial_state = init_state_from_founders(
    key=state_key,
    founder_pop=founder_pop_with_pheno,
    sp=sp,
    config=config,
)

print(f"Initial write position: {initial_state.write_pos}")
print(f"Next available ID: {initial_state.next_id}")
print(f"Founder population active: {jnp.sum(initial_state.is_active)}")

# 8. Verification
print("\n--- Verification ---")
active_mask = initial_state.is_active
var_a = jnp.var(initial_state.bv[active_mask])
var_p = jnp.var(initial_state.pheno[active_mask])
realized_h2 = var_a / var_p

print(f"Target narrow-sense heritability (h2): {h2:.4f}")
print(f"Realized additive variance (VarA) in founders: {var_a:.4f}")
print(f"Realized phenotypic variance (VarP) in founders: {var_p:.4f}")
print(f"Realized narrow-sense heritability (h2) in founders: {realized_h2:.4f}")


# ==============================================================================
# --- NEW: Selection and Meiosis Workflow ---
# ==============================================================================

print("\n--- 9. Phenotypic Selection ---")

# In JAX, we operate on the full, fixed-size arrays and use masks.
# For selection, we can set the phenotype of inactive individuals to a very
# low number to ensure they are not selected.
pheno = initial_state.pheno[:, 0]  # Assuming one trait
selectable_pheno = jnp.where(initial_state.is_active, pheno, -jnp.inf)

# Select the top 20% (20 individuals from the 100 active founders)
n_to_select = 20
# Use lax.top_k, which is more efficient than argsort for selection.
_, top_indices = jax.lax.top_k(selectable_pheno, k=n_to_select)

print(f"Selected {len(top_indices)} individuals with top phenotypes.")
print(f"Indices of selected individuals: {top_indices}")


print("\n--- 10. Random Mating ---")

# Randomly shuffle the selected indices to create mating pairs
shuffled_indices = jax.random.permutation(selection_key, top_indices)

# Split the shuffled group into mothers and fathers
n_crosses = n_to_select // 2
mother_indices = shuffled_indices[:n_crosses]
father_indices = shuffled_indices[n_crosses:]

print(f"Created {n_crosses} random pairs for mating.")
print(f"Mother indices: {mother_indices}")
print(f"Father indices: {father_indices}")


print("\n--- 11. Produce Offspring via Meiosis ---")

# Call the top-level, JIT-compatible meiosis kernel
# This will be extremely fast on a GPU as it vmaps over all crosses.
offspring_geno, offspring_ibd = produce_offspring(
    key=meiosis_key,
    state=initial_state,
    sp=sp,
    config=config,
    mother_indices=mother_indices,
    father_indices=father_indices,
)

print("Meiosis complete.")
print(f"Shape of offspring geno array: {offspring_geno.shape}")
print(f"Shape of offspring IBD array: {offspring_ibd.shape}")

# The `offspring_geno` and `offspring_ibd` arrays are now ready to be
# written into the main `SimState` array in the next generation step.

--- Defining Trait ---
Trait added with 100 QTLs.

--- Setting Initial Phenotypes with h2 ---

--- Initializing Simulation State ---
Initial write position: 100
Next available ID: 100
Founder population active: 100

--- Verification ---
Target narrow-sense heritability (h2): 0.4000
Realized additive variance (VarA) in founders: 1.0000
Realized phenotypic variance (VarP) in founders: 2.6278
Realized narrow-sense heritability (h2) in founders: 0.3805

--- 9. Phenotypic Selection ---
Selected 20 individuals with top phenotypes.
Indices of selected individuals: [93 68 29 31 87 34 26 92 71 88 77 33 59 75 30  0 18 52 39 35]

--- 10. Random Mating ---
Created 10 random pairs for mating.
Mother indices: [71 31 52 68 88 75 92 59  0 18]
Father indices: [39 29 35 87 93 77 33 34 26 30]

--- 11. Produce Offspring via Meiosis ---
Meiosis complete.
Shape of offspring geno array: (10, 10, 2, 100)
Shape of offspring IBD array: (10, 10, 2, 100)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()